In [1]:
import rospy
import std_msgs.msg
import rosbag

from towr_gui_kivy.msg import TowrCommand2, TowrCommandSeq
from xpp_msgs.msg import StateLin3d

## Trajectories:

In [2]:
WALK = 0
TROT = 1

# GAIT = TROT
GAIT = WALK

class PublishManager:
    def __init__(self, waypoint_pos_list, waypoint_ang_list,
                total_duration=3, gait_n=9):
        msg = TowrCommandSeq()
        n_waypoints = len(waypoint_pos_list)
        assert n_waypoints == len(waypoint_ang_list)
        for j in range(n_waypoints):
            goal_lin = StateLin3d()
            x, y, z = waypoint_pos_list[j]
            goal_lin.pos.x = x
            goal_lin.pos.y = y
            goal_lin.pos.z = z
            msg.goal_lin_seq.append(goal_lin)

            goal_ang = StateLin3d()
            [x, y, z] = waypoint_ang_list[j]
            goal_ang.pos.x = x
            goal_ang.pos.y = y
            goal_ang.pos.z = z
            msg.goal_ang_seq.append(goal_ang)

        # msg.total_duration = 3          # reach in 3sec
        msg.total_duration = total_duration          # reach in 3sec
        msg.replay_trajectory = False
        msg.replay_speed = 1            # x1.00 speed
        msg.play_initialization = False
        msg.plot_trajectory = False
        msg.optimize = True             # optimize trajectory
        msg.robot = 4                   # 0: monoped, 1: biped, 4: Aliengo
        msg.terrain = 0
        msg.gait = GAIT                 # 0: walk, 1: trot
        msg.gait_n = gait_n                  # default: 6 gait blocks prob?
        # msg.gait_n = 9                  # default: 6 gait blocks prob?
        msg.optimize_phase_durations = False
        # msg.optimize_phase_durations = True

        self.msg = msg

    def set_waypoints(self, waypoints):
        self.msg.goal_lin_seq.clear()
        self.msg.goal_ang_seq.clear()

        for waypoint in waypoints:
            goal_lin = StateLin3d()
            [x,y,z] = waypoint.world_pos
            goal_lin.pos.x = x
            goal_lin.pos.y = y
            goal_lin.pos.z = z
            self.msg.goal_lin_seq.append(goal_lin)

            goal_ang = StateLin3d()
            [x, y, z] = waypoint.world_ang
            goal_ang.pos.x = x
            goal_ang.pos.y = y
            goal_ang.pos.z = z
            self.msg.goal_ang_seq.append(goal_ang)

    def publish(self):
#         print(self.msg)
        pub.publish(self.msg)

def merge_bags(output_bag, n_bags):
    input_bag_list = [f'/home/whitealex95/.ros/towr_trajectory-{i}.bag' for i in range(n_bags)]
    print(f'Start merging {n_bags} bags to {output_bag}')
    with rosbag.Bag(output_bag, 'w') as outbag:
        t_margin = rospy.Duration.from_sec(0)
        for input_bag in input_bag_list:
            for topic, msg, t in rosbag.Bag(input_bag).read_messages():
                outbag.write(topic, msg, t+t_margin)
            t_margin = rospy.Duration.from_sec((t+t_margin).to_sec())
    print('Done')
    

In [3]:
pub = rospy.Publisher('/towr/user_command_seq', TowrCommandSeq, queue_size=1)
# rospy.Subscriber("/towr/ros_vis_traj", std_msgs.msg.String, sub_callback)
rospy.init_node('ros_gui_kivy', anonymous=False)

### Type (1/4) : Tilt
#### Notes
- s1.5, d3, n9 is bad. prefer n7,n8
- generate wrong force when d6_n7 but ok when n8

- Best case: 
- (s, d, n)


- (1, 5, 7)
- (1, 7, 7)

- (1.25, 7, 7)
- (1.25, 7, 8)
- (1.25, 7, 9)

- (1.5, 7, 7)
- (1.5, 8, 8)
- (1.5, 8, 9)

- (1.75, 7, 9)
- (1.75, 8, 8)
- (1.75, 8, 9)

- (2.0, 8.1, 10)

Non-Flip:
- (1.25, 7, 8)
- (1.25, 7, 9)

- (1.5, 7, 7)
- (1.5, 8, 8)
- (1.5, 8, 9)

- (1.75, 7.1, 9)
- (1.75, 8, 8)
- (1.75, 8, 9)

- (2.0, 8.1, 10)


In [20]:
import math
import numpy as np
def cos(x):
    return math.cos(math.pi/180 * x)
def sin(x):
    return math.sin(math.pi/180 * x)

scale = 1.25
PI = 3.141592
flip = False
# start: pos=[0,0,0], ang=[0,0,0]

tilt_angles = np.array([0, 0, 15, 30, 45, 60, 90, 90, 75, 60, 45, 30, 0])
if flip:
    tilt_angles *= -1
WP_pos = []
WP_ang = []

last_pos = np.array([0,0,0])
for ang in tilt_angles:
    cur_pos = last_pos + np.array([cos(ang), sin(ang), 0])
    WP_pos.append(cur_pos)
    WP_ang.append(np.zeros(3))
    last_pos = cur_pos

WP_pos = np.array(WP_pos) * scale
WP_ang = np.array(WP_ang) * scale
print(WP_pos, WP_ang)

seq_name = 'style1-tilt'
if scale != 1:
    seq_name += f'-s{scale}'
if flip:
    seq_name += f'-flip'

v = 1
d = 7
n = 7

pm = PublishManager(WP_pos, WP_ang, total_duration=d, gait_n = n)
pm.publish()


[[ 1.25        0.          0.        ]
 [ 2.5         0.          0.        ]
 [ 3.70740728  0.32352381  0.        ]
 [ 4.78993904  0.94852381  0.        ]
 [ 5.67382251  1.83240728  0.        ]
 [ 6.29882251  2.91493904  0.        ]
 [ 6.29882251  4.16493904  0.        ]
 [ 6.29882251  5.41493904  0.        ]
 [ 6.62234632  6.62234632  0.        ]
 [ 7.24734632  7.70487808  0.        ]
 [ 8.1312298   8.58876155  0.        ]
 [ 9.21376155  9.21376155  0.        ]
 [10.46376155  9.21376155  0.        ]] [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [19]:
out_folder = '/home/whitealex95/.ros/'
out_bag_name = f'towr_trajectory/{seq_name}_v{v}_d{d}_n{n}.bag'
merge_bags(out_folder+out_bag_name, len(WP_pos))

Start merging 13 bags to /home/whitealex95/.ros/towr_trajectory/style1-tilt-s1.25_v1_d7_n8.bag
Done


### Type (2/4) : Curve
#### Notes
- s1.5, d3, n9 is bad. prefer n7,n8
- generate wrong force when d6_n7 but ok when n8

- Best case: 
- (s, d, n)
- (1, 6, 7)

- (1.25, 6, 7)
- (1.25, 6, 8)

- (1.5, 6, 8)
- (1.5, 6, 7)

- (1.75, 6, 8)
- (1.75, 6, 9)

- (2.0, 6, 8)
- (2.0, 6, 9)
- (2.5, 6, 10)
- (2.75, 6, 10)

- (3.0, 6, 10)
- (3.0, 7, 10)

Non-flip:
- (1, 6, 8)

- (1.25, 6, 7)

- (1.5, 6, 8)
- (1.5, 6, 7)

- (1.75, 6, 7)
- (1.75, 6, 8)
- (1.75, 6, 9)

- (2.0, 6, 8)
- (2.0, 6, 9)

- (2.5, 6, 10)
- (2.75, 6, 10)

- (3.0, 6.1, 10)


In [80]:
import math
import numpy as np
def cos(x):
    return math.cos(math.pi/180 * x)
def sin(x):
    return math.sin(math.pi/180 * x)
def deg2rad(x):
    return math.pi/180 * x
scale = 3.0
flip = False
# start: pos=[0,0,0], ang=[0,0,0]

tilt_angles = np.array([90, 75, 45, 0, -45, -75, 0])
if flip:
    tilt_angles *= -1
WP_pos = []
WP_ang = []

last_pos = np.array([0,0,0])
last_ang = 0
for ang in tilt_angles:
    cur_pos = last_pos + np.array([cos(last_ang) + cos(ang), sin(last_ang) + sin(ang), 0])/2
    WP_pos.append(cur_pos)
    WP_ang.append(np.array([0, 0, deg2rad(ang)]))
    last_pos = cur_pos
    last_ang = ang

WP_pos = np.array(WP_pos) * scale
WP_ang = np.array(WP_ang)# * scale
print(WP_pos, WP_ang)

seq_name = 'style2-curve'
if scale != 1:
    seq_name += f'-s{scale}'
if flip:
    seq_name += f'-flip'

v = 1
d = 6.1
n = 10

pm = PublishManager(WP_pos, WP_ang, total_duration=d, gait_n = n)
pm.publish()


[[ 1.5         1.5         0.        ]
 [ 1.88822857  4.44888874  0.        ]
 [ 3.33711731  6.95843765  0.        ]
 [ 5.89777748  8.01909782  0.        ]
 [ 8.45843765  6.95843765  0.        ]
 [ 9.90732639  4.44888874  0.        ]
 [11.79555496  3.          0.        ]] [[ 0.          0.          1.57079633]
 [ 0.          0.          1.30899694]
 [ 0.          0.          0.78539816]
 [ 0.          0.          0.        ]
 [ 0.          0.         -0.78539816]
 [ 0.          0.         -1.30899694]
 [ 0.          0.          0.        ]]


In [81]:
out_folder = '/home/whitealex95/.ros/'
out_bag_name = f'towr_trajectory/{seq_name}_v{v}_d{d}_n{n}.bag'
merge_bags(out_folder+out_bag_name, len(WP_pos))

Start merging 7 bags to /home/whitealex95/.ros/towr_trajectory/style2-curve-s3.0_v1_d6.1_n10.bag
Done


### Type (3/4) : Spin
#### Notes
- s1.5, d3, n9 is bad. prefer n7,n8
- generate wrong force when d6_n7 but ok when n8

- Best case: 
- (s, d, n)
- (1, 5, 9)
- (1, 5.1, 9)
- (1, 6.5, 11)
- (1, 8, 12)

Flip:
- (1, 4, 8)
- (1, 5, 9)
- (1, 5.1, 9)
- (1, 6.5, 11)
- (1, 8, 12)


In [105]:
import math
import numpy as np
def cos(x):
    return math.cos(math.pi/180 * x)
def sin(x):
    return math.sin(math.pi/180 * x)
def deg2rad(x):
    return math.pi/180 * x
scale = 1
flip = False
# start: pos=[0,0,0], ang=[0,0,0]

tilt_angles = np.array([90, 0, 180, 0])
if flip:
    tilt_angles *= -1
WP_pos = []
WP_ang = []

last_pos = np.array([0,0,0])
last_ang = 0
for ang in tilt_angles:
#     cur_pos = last_pos + np.array([cos(last_ang) + cos(ang), sin(last_ang) + sin(ang), 0])/2
    WP_pos.append(np.zeros(3))
    WP_ang.append(np.array([0, 0, deg2rad(ang)]))
    last_pos = cur_pos
    last_ang = ang

WP_pos = np.array(WP_pos) * scale
WP_ang = np.array(WP_ang)# * scale
print(WP_pos, WP_ang)

seq_name = 'style3-spin'
if scale != 1:
    seq_name += f'-s{scale}'
if flip:
    seq_name += f'-flip'

v = 1
d = 5
n = 9

pm = PublishManager(WP_pos, WP_ang, total_duration=d, gait_n = n)
pm.publish()


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] [[0.         0.         1.57079633]
 [0.         0.         0.        ]
 [0.         0.         3.14159265]
 [0.         0.         0.        ]]


In [99]:
out_folder = '/home/whitealex95/.ros/'
out_bag_name = f'towr_trajectory/{seq_name}_v{v}_d{d}_n{n}.bag'
merge_bags(out_folder+out_bag_name, len(WP_pos))

Start merging 4 bags to /home/whitealex95/.ros/towr_trajectory/style3-spin-flip_v1_d4_n8.bag
Done


### Type (4/4) :BackTurn
#### Notes
- s1.5, d3, n9 is bad. prefer n7,n8
- generate wrong force when d6_n7 but ok when n8

- Best case: 
- (s, d, n)
- (1, 5.2, 9)
- (1, 5.2, 10)
- (1, 6.2, 10)
- (1, 7, 12)
- (1, 8, 13)

### Flipped:
- (1, 5.2, 10)
- (1, 6, 10)
- (1, 7, 11)
- (1, 8, 12)


In [136]:
import math
import numpy as np
def cos(x):
    return math.cos(math.pi/180 * x)
def sin(x):
    return math.sin(math.pi/180 * x)
def deg2rad(x):
    return math.pi/180 * x
scale = 1
flip = False
# start: pos=[0,0,0], ang=[0,0,0]

tilt_angles = np.array([45, 0, 180, 0, 90, 0])
# tilt_angles = np.array([45, 0, ])#90, 0, 135, 180])
if flip:
    tilt_angles *= -1
WP_pos = []
WP_ang = []

last_pos = np.array([0,0,0])
last_ang = 0
for ang in tilt_angles:
    cur_pos = np.array([-1 + cos(ang), sin(ang), 0])
    WP_pos.append(cur_pos)
    WP_ang.append(np.array([0, 0, deg2rad(ang)]))
    last_pos = cur_pos
    last_ang = ang

WP_pos = np.array(WP_pos) * scale
WP_ang = np.array(WP_ang)# * scale
print(WP_pos, WP_ang)

seq_name = 'style4-backturn'
if scale != 1:
    seq_name += f'-s{scale}'
if flip:
    seq_name += f'-flip'

v = 1
d = 8
n = 13

pm = PublishManager(WP_pos, WP_ang, total_duration=d, gait_n = n)
pm.publish()


[[-2.92893219e-01  7.07106781e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.00000000e+00  1.22464680e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]] [[0.         0.         0.78539816]
 [0.         0.         0.        ]
 [0.         0.         3.14159265]
 [0.         0.         0.        ]
 [0.         0.         1.57079633]
 [0.         0.         0.        ]]


In [137]:
out_folder = '/home/whitealex95/.ros/'
out_bag_name = f'towr_trajectory/{seq_name}_v{v}_d{d}_n{n}.bag'
merge_bags(out_folder+out_bag_name, len(WP_pos))

Start merging 6 bags to /home/whitealex95/.ros/towr_trajectory/style4-backturn_v1_d8_n13.bag
Done


In [ ]:
### Type (4+/4+) :Left Right
#### Notes
- s1.5, d3, n9 is bad. prefer n7,n8
- generate wrong force when d6_n7 but ok when n8

- Best case: 
- (s, d, n)
- (1.25, 7, 7)
- (1.25, 7, 8)
- (1.25, 7, 9)
- (1.5, 7, 9)
- (1.75, 7, 10)

### Flipped:
- (1.25, 7, 9)
- (1.25, 7, 10)
- (1.5, 7, 10)
- (1.75, 7.1, 10)


In [169]:
import math
import numpy as np
def cos(x):
    return math.cos(math.pi/180 * x)
def sin(x):
    return math.sin(math.pi/180 * x)

scale = 1
PI = 3.141592
flip = False
# start: pos=[0,0,0], ang=[0,0,0]

tilt_angles = np.array([90, 90, 90])
if flip:
    tilt_angles *= -1
WP_pos = []
WP_ang = []

last_pos = np.array([0,0,0])
for ang in tilt_angles:
    cur_pos = last_pos + np.array([cos(ang), sin(ang), 0])
    WP_pos.append(cur_pos)
    WP_ang.append(np.zeros(3))
    last_pos = cur_pos

WP_pos = np.array(WP_pos) * scale
WP_ang = np.array(WP_ang) * scale
print(WP_pos, WP_ang)

seq_name = 'style5-leftright'
if scale != 1:
    seq_name += f'-s{scale}'
if flip:
    seq_name += f'-flip'

v = 1
d = 7
n = 7

pm = PublishManager(WP_pos, WP_ang, total_duration=d, gait_n = n)
pm.publish()


[[6.1232340e-17 1.0000000e+00 0.0000000e+00]
 [1.2246468e-16 2.0000000e+00 0.0000000e+00]
 [1.8369702e-16 3.0000000e+00 0.0000000e+00]] [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [166]:
out_folder = '/home/whitealex95/.ros/'
out_bag_name = f'towr_trajectory/{seq_name}_v{v}_d{d}_n{n}.bag'
merge_bags(out_folder+out_bag_name, len(WP_pos))

Start merging 3 bags to /home/whitealex95/.ros/towr_trajectory/style5-leftright-s1.25-flip_v1_d7_n9.bag
Done
